<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/LSTM_Module2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
import pandas as pd
import requests
from itertools import islice
import numpy as np
import xml.etree.ElementTree as ET
import random
import tensorflow as tf
from scipy.sparse import csr_matrix
import tensorflow as tf
from keras.preprocessing import sequence
from keras.preprocessing import text
from tensorflow import keras
from keras.models import Sequential
from keras import models
from keras.layers import *
from keras.models import *
from keras import backend as K
from keras import initializers
from keras import regularizers
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Attention, Flatten, Activation, RepeatVector, Permute, Multiply
from keras.layers import CuDNNLSTM
from keras.layers import Dropout, Multiply, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [137]:
import pandas as pd
df = pd.read_csv("/content/gdrive/MyDrive/1-restaurant-train.csv", delimiter = '\t', header = None )
df.columns = ['Rating', 'text']
df.reset_index(drop=True, inplace=True)
df.dropna(axis = 0, inplace = True)

In [138]:
df = df.sample(frac =.1)
df['text'] = df['text'].astype(str)

In [139]:
df_category = pd.get_dummies(df['Rating'])

In [140]:
df_category.head()

,1,2,3,4,5
34006,0,1,0,0,0
71792,0,1,0,0,0
44373,0,0,0,0,1
56443,0,0,0,0,1
71164,0,0,0,1,0


In [141]:
df['Rating'].unique()

array([2, 5, 4, 3, 1])

In [142]:
df.head()

,Rating,text
34006,2,"""Just another sushi joint that will never live..."
71792,2,"""We visited Harlow's around 9am on a Monday mo..."
44373,5,Oh my god! five stars only because of their wa...
56443,5,"""A prospective customer at Sakana must check o..."
71164,4,Excellent. Bread was unremarkable. Not tasty ...


In [143]:
data = df['text']
data = list(data)
labels = df_category.iloc[:, :5]


In [144]:
labels

,1,2,3,4,5
34006,0,1,0,0,0
71792,0,1,0,0,0
44373,0,0,0,0,1
56443,0,0,0,0,1
71164,0,0,0,1,0
...,...,...,...,...,...
19134,0,0,0,0,1
24620,0,0,1,0,0
80254,0,0,0,1,0
6524,0,0,0,0,1


In [145]:
def split_training_and_validation_sets(data, labels, validation_split):

    num_training_samples = int((1 - validation_split) * len(data))
    return ((data[:num_training_samples], labels[:num_training_samples]),
            (data[num_training_samples:], labels[num_training_samples:]))
    
(train_data, train_label), (valid_data, valid_label) = split_training_and_validation_sets(data, labels, 0.1)


In [146]:
train_label = tf.convert_to_tensor(train_label, dtype=tf.float32)
valid_label = tf.convert_to_tensor(valid_label, dtype=tf.float32)

In [147]:
def sequence_vectorizer(train_data, valid_data):

    
    tokenizer = text.Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(valid_data)
    

    x_train = tokenizer.texts_to_sequences(train_data)
    x_val = tokenizer.texts_to_sequences(valid_data)
    

    x_train = sequence.pad_sequences(x_train, maxlen=100)
    x_val = sequence.pad_sequences(x_val, maxlen=100)
    

    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)

    return x_train, x_val, tokenizer.word_index

In [148]:
x_train, x_val, word_index = sequence_vectorizer(train_data, valid_data)

In [73]:
!unzip "/content/gdrive/MyDrive/glove.42B.300d.zip" -d "/content"

Archive:  /content/gdrive/MyDrive/glove.42B.300d.zip
replace /content/glove.42B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [149]:
glove_file = "glove.42B.300d.txt"
import tqdm

EMBEDDING_VECTOR_LENGTH = 200 # <=200
def construct_embedding_matrix(glove_file, word_index):
    embedding_dict = {}
    with open(glove_file,'r') as f:
        for line in f:
            values=line.split()
            # get the word
            word=values[0]
            if word in word_index.keys():
                # get the vector
                vector = np.asarray(values[1:], 'float32')
                embedding_dict[word] = vector
    ###  oov words (out of vacabulary words) will be mapped to 0 vectors

    num_words=len(word_index)+1
    #initialize it to 0
    embedding_matrix=np.zeros((num_words, EMBEDDING_VECTOR_LENGTH))

    for word,i in tqdm.tqdm(word_index.items()):
        if i < num_words:
            vect=embedding_dict.get(word, [])
            if len(vect)>0:
                embedding_matrix[i] = vect[:EMBEDDING_VECTOR_LENGTH]
    return embedding_matrix
  
embedding_matrix =  construct_embedding_matrix(glove_file, word_index)

100%|██████████| 11540/11540 [00:00<00:00, 396363.03it/s]


In [150]:
def LSTM_Model():
  model = models.Sequential()
  model.add(Embedding(len(word_index) + 1,
                            EMBEDDING_VECTOR_LENGTH,
                            weights=[embedding_matrix],
                            input_length=100,
                            trainable=False))
  model.add(SpatialDropout1D(0.4))
  model.add(CuDNNLSTM(176, return_sequences=True))
  model.add(Dropout(0.8))
  model.add(CuDNNLSTM(64, return_sequences=True))
  model.add(Dropout(0.8))
  model.add(CuDNNLSTM(32))
  model.add(Dense(5,activation='sigmoid'))
  return model

In [151]:
model = LSTM_Model()
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 100, 200)          2308200   
                                                                 
 spatial_dropout1d_14 (Spati  (None, 100, 200)         0         
 alDropout1D)                                                    
                                                                 
 cu_dnnlstm_42 (CuDNNLSTM)   (None, 100, 176)          266112    
                                                                 
 dropout_28 (Dropout)        (None, 100, 176)          0         
                                                                 
 cu_dnnlstm_43 (CuDNNLSTM)   (None, 100, 64)           61952     
                                                                 
 dropout_29 (Dropout)        (None, 100, 64)           0         
                                                     

In [153]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8)]
history = model.fit(x_train, train_label,
            epochs=100,
            callbacks = callbacks,
            validation_data=(x_val, valid_label),
            verbose=2,
            batch_size=64)

Epoch 1/100
116/116 - 7s - loss: 0.4296 - accuracy: 0.3798 - val_loss: 0.4160 - val_accuracy: 0.3922 - 7s/epoch - 61ms/step
Epoch 2/100
116/116 - 7s - loss: 0.4199 - accuracy: 0.3928 - val_loss: 0.4103 - val_accuracy: 0.4202 - 7s/epoch - 57ms/step
Epoch 3/100
116/116 - 6s - loss: 0.4135 - accuracy: 0.4065 - val_loss: 0.4036 - val_accuracy: 0.4373 - 6s/epoch - 56ms/step
Epoch 4/100
116/116 - 6s - loss: 0.4079 - accuracy: 0.4229 - val_loss: 0.4015 - val_accuracy: 0.4458 - 6s/epoch - 56ms/step
Epoch 5/100
116/116 - 6s - loss: 0.4051 - accuracy: 0.4306 - val_loss: 0.3992 - val_accuracy: 0.4580 - 6s/epoch - 56ms/step
Epoch 6/100
116/116 - 6s - loss: 0.3977 - accuracy: 0.4447 - val_loss: 0.3999 - val_accuracy: 0.4361 - 6s/epoch - 56ms/step
Epoch 7/100
116/116 - 7s - loss: 0.3909 - accuracy: 0.4700 - val_loss: 0.3904 - val_accuracy: 0.4909 - 7s/epoch - 56ms/step
Epoch 8/100
116/116 - 6s - loss: 0.3874 - accuracy: 0.4762 - val_loss: 0.3954 - val_accuracy: 0.4568 - 6s/epoch - 56ms/step
Epoch 9/